In [2]:
import numpy as np
import pandas as pd
import h5py
import sys
import matplotlib.pyplot as plt
from statistics import mean
from scipy.optimize import curve_fit

sys.path.insert(0, './radiometric')
from LUT_functions import pixel_registration, create_LUT_df
import scipy

# Seaborn for plotting and styling
import seaborn as sns
sns.set_theme()
sns.set_style("whitegrid", {'axes.grid' : False})

In [3]:
#choose filepath to desired data
path = '/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_data/BBpolarized/'
cal_file1 =  "/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_files/cam1pixel.h5"
cal_file2 =  "/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_files/cam2pixel.h5"

In [4]:
#will create a df and print the names of the keys in the original hdf5 file
df1 = create_LUT_df(path,'cam1_0deg.h5')
df2 = create_LUT_df(path,'cam2_0deg.h5')

In [5]:
waves= np.linspace(7,12,61)
lps = [0,10,20,30,40,50,60,70,80,90,110,130,150,170]
angles = np.round(np.linspace(-7.07,7.07,35),1)

In [6]:
#import nuc calibration
sys.path.insert(0, './NUC')
from NUC_functions import import_NUC, apply_NUC
cal_path = '/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_files/NUC.h5'

m1,b1,T_ref1,m2,b2,T_ref2 = import_NUC(cal_path)

In [ ]:
#apply pixel registration and NUC to full dataset

ymin1 = 105; ymax1 = 140;
ymin2 = 130; ymax2 = 165;

mean_ims1 =[]
mean_ims2 =[]
std_ims1  =[]
std_ims2  =[]

mean_ims1n =[]
mean_ims2n =[]
std_ims1n  =[]
std_ims2n  =[]

for i in lps:
    #will create a df and print the names of the keys in the original hdf5 file
    df1 = create_LUT_df(path,'cam1_'+str(i)+'deg.h5')
    df2 = create_LUT_df(path,'cam2_'+str(i)+'deg.h5')
    
    corrected_images1= [] ; corrected_images1n= [];
    corrected_images2= [] ; corrected_images2n= []

    for i in range(len(df1)):
        #first without NUC correction
        r = df1['images1'][i];
        c = pixel_registration(r,cal_file1,waves,ymin1,ymax1)
        
        #now with NUC correction
        T = df1['temp1'][i];
        rn = apply_NUC(r,T,m1,b1,T_ref1)
        cn = pixel_registration(rn,cal_file1,waves,ymin1,ymax1)
        
        corrected_images1.append(c)
        corrected_images1n.append(cn)

    for i in range(len(df2)):
        #first without NUC correction
        r = df2['images1'][i];
        c = pixel_registration(r,cal_file2,waves,ymin2,ymax2)
        
        #now with NUC correction
        T = df2['temp1'][i];
        rn = apply_NUC(r,T,m2,b2,T_ref2)
        cn = pixel_registration(rn,cal_file2,waves,ymin2,ymax2)
        
        corrected_images2.append(c)
        corrected_images2n.append(cn)

    mean_ims1.append(np.mean(corrected_images1,axis = 0))
    mean_ims1n.append(np.mean(corrected_images1n,axis = 0))
    mean_ims2.append(np.mean(corrected_images2,axis = 0))
    mean_ims2n.append(np.mean(corrected_images2n,axis = 0))


    std_ims1.append(np.std(corrected_images1,axis = 0))
    std_ims2.append(np.std(corrected_images2,axis = 0))
    std_ims1n.append(np.std(corrected_images1n,axis = 0))
    std_ims2n.append(np.std(corrected_images2n,axis = 0))

In [ ]:
d = {'lps': lps, 'ims1': list(mean_ims1),'s1': list(std_ims1), 'ims2':list(mean_ims2),'s2': list(std_ims2)}
dn = {'lps': lps, 'ims1': list(mean_ims1n),'s1': list(std_ims1n), 'ims2':list(mean_ims2n),'s2': list(std_ims2n)}


In [ ]:
df = pd.DataFrame.from_dict(d, orient='index')
df = df.transpose()

dfn = pd.DataFrame.from_dict(dn, orient='index')
dfn = dfn.transpose()

dfn

In [ ]:

df.to_hdf('/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_data/LUT/df_pol.h5', key='s')
dfn.to_hdf('/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_data/LUT/df_pol_NUC.h5', key='s')


In [ ]:
def finv(y,A,B):
    return ((y-B)/A)**0.5


In [ ]:
#import radiometric data

#choose filepath to desired data
save_path =   '/Users/kirahart/Dropbox/GitHub/IRCSP2_data/calibration_files/'
name =  "LUT.h5" 

hf = h5py.File(save_path+name, 'r')
B2 = hf.get('B2')
B1 = hf.get('B1')

In [ ]:

j = 10;
I1 = df['ims1'][j]-B1
I2 = df['ims2'][j]-B2
plt.matshow((I1-I2)/(I1+I2),cmap = 'inferno' )
plt.colorbar()
plt.ylabel('Angular FOV [$^\circ$]')
plt.xlabel('Wavelength [$\mu m$]')
plt.ax = plt.gca()
plt.ax.xaxis.set_ticks_position('bottom')
plt.xticks([0, 20, 40,60], [waves[0],round(waves[20]),waves[30],waves[60]])
plt.yticks([0, 9,17,26,34], [angles[0], angles[9],angles[17],angles[26],angles[34]])
plt.title('Modulation Function, AOLP = ' + str(lps[j])+ '$ ^\circ $ ')
plt.clim([-1,1])
plt.show()


In [ ]:
#create sum column
df["totalrad"] = df['ims2']+df['ims1']
df["totalstd"] = (df['s2']**2+df['s1']**2)**0.5

In [ ]:
cmin = np.min(list(df['totalrad'])); cmax=np.max(list(df['totalrad']))

j = 5;
plt.matshow(df['totalrad'][j],cmap = 'inferno' )
plt.colorbar()
plt.ylabel('Spatial Sampling')
plt.xlabel('Wavelength [$\mu m$]')
plt.ax = plt.gca()
plt.ax.xaxis.set_ticks_position('bottom')
plt.xticks([0, 20, 40,60], [waves[0],round(waves[20]),waves[30],waves[60]])
plt.title('AOLP = ' + str(lps[j])+ '$^\circ $ \n no NUC')
#plt.clim(cmin,cmax)
plt.show()

In [ ]:
# pixel fit 
def f(x, A, B): # this is your 'straight line' y=f(x)
    return A*x**2 + B

def finv(y,A,B):
    return ((y-B)/A)**0.5


In [ ]:
x = 10
y = 30
plt.errorbar(dfn['lps'],get_slice(dfn['ims2'],x,y),label = 'no NUC')
#plt.errorbar(df['lps'],get_slice_avg(df['ims1'],x),yerr=get_slice_avg(df['s1'],x),label = 'no NUC')
plt.title('spatially averaged response \n $\lambda = $' + str(round(waves[x],2)) + ' $  \mu m$')
plt.xlabel('Scene Temperature [$^\circ C$ ] ')
plt.ylabel('avg camera count')
plt.legend()
plt.show()


In [ ]:
def get_slice(df,x,y):
    sli = []
    for i in range(len(df)):
        s = df[i]
        sli.append(s[y,x])
    return(sli)

def get_slice_avg(df,x):
    sli = []
    for i in range(len(df)):
        s = df[i]
        sli.append(np.mean(s[:,x]))
    return(sli)

In [ ]:
''''playing with off set '''

j = 10;
I1 = df['ims1'][j]-B1
I2 = df['ims2'][j]-B2
plt.matshow(I1,cmap = 'inferno' )
plt.colorbar()
plt.ylabel('Angular FOV [$^\circ$]')
plt.xlabel('Wavelength [$\mu m$]')
plt.ax = plt.gca()
plt.ax.xaxis.set_ticks_position('bottom')
plt.xticks([0, 20, 40,60], [waves[0],round(waves[20]),waves[30],waves[60]])
plt.yticks([0, 9,17,26,34], [angles[0], angles[9],angles[17],angles[26],angles[34]])
plt.title('Modulation Function, AOLP = ' + str(lps[j])+ '$ ^\circ $ ')
plt.clim([-1,1])
plt.show()
